# Сравнение LSTM и Transformer моделей для автодополнения текста

In [ ]:
import os
import torch
import numpy as np
from torch.utils.data import DataLoader
import torch.nn as nn
import evaluate
import urllib.request
from tqdm import tqdm
import matplotlib.pyplot as plt

from src import data_utils, split_dataset, next_token_dataset, lstm_model, transformer


TOKENIZER = next_token_dataset.TOKENIZER
MIN_LEN = next_token_dataset.MIN_LEN
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Используемое устройство: {DEVICE}")
print(f"Размер словаря: {len(TOKENIZER)}")

## Подготовка данных

In [ ]:
def read_texts_from_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [line.strip() for line in f if line.strip()]


if not (os.path.exists("data/train.txt") and os.path.exists("data/val.txt") and os.path.exists("data/test.txt")):
    print("Файлы данных не найдены, загружаем и обрабатываем исходные данные...")

    
    url = 'https://code.s3.yandex.net/deep-learning/tweets.txt'
    filename = "data/tweets.txt"
    if not os.path.exists(filename):
        os.makedirs("data", exist_ok=True)
        urllib.request.urlretrieve(url, filename)
        print(f"Загружен файл: {filename}")
    
    print("Очистка текста...")
    with open(filename, 'r', encoding='utf-8') as infile, \
         open("data/cleaned_text.txt", "w", encoding="utf-8") as outfile:
        for line in infile:
            if not line.strip():
                continue
            cleaned_line = data_utils.clean_text(line)
            if cleaned_line:
                outfile.write(cleaned_line + '\n')
    print("Очистка завершена.")
    
    print("Разделение на выборки...")
    with open("data/cleaned_text.txt", "r", encoding="utf-8") as f:
        lines = [line.strip() for line in f.readlines() if line.strip()]
    
    splits = split_dataset.split_dataset(lines)
    train_texts = splits["train"]
    val_texts = splits["val"] 
    test_texts = splits["test"]
    
else:
    print("Файлы данных найдены, загружаем готовые выборки...")
    train_texts = read_texts_from_file("data/train.txt")
    val_texts = read_texts_from_file("data/val.txt")
    test_texts = read_texts_from_file("data/test.txt")

print(f"Загружено: train({len(train_texts)}), val({len(val_texts)}), test({len(test_texts)})")

train_dataset = next_token_dataset.TextDataset(train_texts, num_targets=1)
val_dataset = next_token_dataset.TextDataset(val_texts, num_targets=1)
test_dataset = next_token_dataset.TextDataset(test_texts, num_targets=1)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, collate_fn=next_token_dataset.collate)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, collate_fn=next_token_dataset.collate)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, collate_fn=next_token_dataset.collate)

print(f"Данные загружены: train({len(train_dataset)}), val({len(val_dataset)}), test({len(test_dataset)})")

## Обучение LSTM модели

In [ ]:
vocab_size = len(TOKENIZER)
lstm_model_instance = lstm_model.LstmModel(
    vocab_size=vocab_size,
    embedding_dim=128,
    hidden_dim=128,
    num_layers=2
)

print("Модель LSTM создана успешно")

In [ ]:
for batch in train_loader:
    contexts = batch['contexts']
    lengths = batch['lengths']
    
    with torch.no_grad():
        logits = lstm_model_instance(contexts, lengths)
    
    print(f"Входной тензор: {contexts.shape}")
    print(f"Выходной тензор (логиты): {logits.shape}")
    print(f"Размер словаря: {logits.size(-1)}")
    break



In [ ]:
rouge_metric = evaluate.load('rouge')
print("ROUGE метрика загружена")

In [ ]:
print("Начало обучения LSTM модели...")
train_losses, val_losses, val_accuracies, val_rouge_scores = lstm_model_instance.train_model(
    n_epochs=3,
    learning_rate=0.001,
    train_loader=train_loader,
    val_loader=val_loader,
    rouge_metric=rouge_metric
)

rouge2 равны нулю поскольку мы учимся предсказывать только один токен, а rouge2 используется для биграмм

In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Функция потерь')
plt.xlabel('Эпоха')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(val_accuracies)
plt.title('Точность')
plt.xlabel('Эпоха')
plt.ylabel('Accuracy')

plt.subplot(1, 3, 3)
rouge_scores_plot = [score['rouge1'] if score else 0 for score in val_rouge_scores]
plt.plot(rouge_scores_plot)
plt.title('ROUGE-1')
plt.xlabel('Эпоха')
plt.ylabel('Score')

plt.tight_layout()
plt.show()

In [ ]:
os.makedirs('models', exist_ok=True)
lstm_model_instance.save_model("models/lstm_model.pth")
print("Модель сохранена в models/lstm_model.pth")

In [ ]:
print("Тестирование LSTM модели...")
criterion = nn.CrossEntropyLoss()
test_loss, test_accuracy, test_rouge = lstm_model_instance.evaluate_model(test_loader, criterion, rouge_metric)
print(f"LSTM Test Loss: {test_loss:.4f}")
print(f"LSTM Test Accuracy: {test_accuracy:.4f}")
if test_rouge:
    print("LSTM Test ROUGE:")
    for metric, score in test_rouge.items():
        print(f"  {metric}: {score:.4f}")

В задании нужно предсказывать 1/4 часть текста, но поскольку предложения имеют разную длину, а в батче разное количество слов для предсказания, то будем использовать фиксированное число слов например 4.

In [ ]:
NUM_WORDS = 4

In [ ]:
def prepare_multiword_prediction_data(texts, num_words=3):
    """Подготовка данных для предсказания нескольких слов"""
    contexts = []
    targets = []
    full_texts = []
    
    for text in texts:
        words = text.split()
        if len(words) < MIN_LEN + num_words:
            continue
        ctx_len = len(words) - num_words
        context = ' '.join(words[:ctx_len])
        target = ' '.join(words[ctx_len:])
        contexts.append(context)
        targets.append(target)
        full_texts.append(context + " " + target)
    
    return contexts, targets, full_texts

In [ ]:
print("Подготовка данных для предсказания 3 слов...")
lstm_contexts, lstm_targets, full_texts = prepare_multiword_prediction_data(test_texts, NUM_WORDS)
print(f"Подготовлено {len(lstm_contexts)} примеров")

print("Предсказание LSTM с авторегрессией...")
lstm_predictions = [""] * len(lstm_contexts)
current_contexts = lstm_contexts.copy()

for word_idx in range(NUM_WORDS):
    print(f"Генерация слова {word_idx + 1}...")
    batch_predictions = []
    
    for i in range(0, len(current_contexts), 32):
        batch_contexts = current_contexts[i:i+32]
        
        for context in batch_contexts:
            try:
                context_ids = torch.tensor(TOKENIZER.encode(context, add_special_tokens=False))
                generated_tokens = lstm_model_instance.generate_tokens(
                    context_ids, max_length=1, temperature=0.8
                )
                pred_word = TOKENIZER.decode(generated_tokens[:1], skip_special_tokens=True)
                batch_predictions.append(pred_word if pred_word else "word")
            except:
                batch_predictions.append("word")
    
    current_contexts = [ctx + " " + pred for ctx, pred in zip(current_contexts, batch_predictions)]
    lstm_predictions = [prev + " " + pred for prev, pred in zip(lstm_predictions, batch_predictions)]

In [ ]:
try:
    lstm_rouge = rouge_metric.compute(
        predictions=lstm_predictions,
        references=lstm_targets,
        use_stemmer=True
    )
    print("\nLSTM ROUGE метрики (предсказание 4 слов):")
    for metric, score in lstm_rouge.items():
        print(f"  {metric}: {score:.4f}")
except Exception as e:
    print(f"Ошибка расчета ROUGE для LSTM: {e}")

## Использование предобученного трансформера

In [ ]:
print("Загрузка предобученного трансформера distilgpt2...")
transformer_model_instance = transformer.TransformerGenerator('distilgpt2')
print("Трансформер загружен успешно")

In [ ]:
print("\nПредсказание трансформера с авторегрессией...")
transformer_predictions = [""] * len(lstm_contexts)
current_contexts_transformer = lstm_contexts.copy()

In [ ]:
for word_idx in range(NUM_WORDS):
    print(f"Генерация слова {word_idx + 1} трансформером...")
    batch_preds = transformer_model_instance.generate(
        current_contexts_transformer, 
        max_new_tokens=1
    )
    
    processed_preds = []
    for i, (pred, context) in enumerate(zip(batch_preds, current_contexts_transformer)):
        if pred.startswith(context):
            new_part = pred[len(context):].strip()
            first_word = new_part.split()[0] if new_part.split() else "word"
            processed_preds.append(first_word)
        else:
            first_word = pred.split()[0] if pred.split() else "word"
            processed_preds.append(first_word)
    
    current_contexts_transformer = [ctx + " " + pred for ctx, pred in zip(current_contexts_transformer, processed_preds)]
    transformer_predictions = [prev + " " + pred for prev, pred in zip(transformer_predictions, processed_preds)]

In [ ]:
try:
    transformer_rouge = rouge_metric.compute(
        predictions=transformer_predictions,
        references=lstm_targets,
        use_stemmer=True
    )
    print("\nТрансформер ROUGE метрики (предсказание 4 слов):")
    for metric, score in transformer_rouge.items():
        print(f"  {metric}: {score:.4f}")
except Exception as e:
    print(f"Ошибка расчета ROUGE для трансформера: {e}")

## Формулирование выводов

In [ ]:
# Финальное сравнение
print("\n" + "="*50)
print("ФИНАЛЬНОЕ СРАВНЕНИЕ (предсказание 4 слов)")
print("="*50)

print(f"\nLSTM модель:")
if 'lstm_rouge' in locals():
    for metric, score in lstm_rouge.items():
        print(f"  {metric}: {score:.4f}")

print(f"\nТрансформер (distilgpt2):")
if 'transformer_rouge' in locals():
    for metric, score in transformer_rouge.items():
        print(f"  {metric}: {score:.4f}")

In [ ]:
# Примеры предсказаний
print("\nПРИМЕРЫ ПРЕДСКАЗАНИЙ:")
print("="*40)
for i in range(min(3, len(lstm_contexts))):
    print(f"\nПример {i+1}:")
    print(f"Контекст: {lstm_contexts[i]}")
    print(f"Референс: {lstm_targets[i]}")
    if i < len(lstm_predictions):
        print(f"LSTM:     {lstm_predictions[i].strip()}")
    if i < len(transformer_predictions):
        print(f"Трансформер: {transformer_predictions[i].strip()}")

## Выводы

- Предсказания трансформера более осмысленные, а LSTM выдает бессмысленные словосочентания.
- Трансформер лучше улавливает смысл и пытается логически его продолжить, теряет смысл и генерирует случайные слова (фразы).
- По численным метрикам хоть и низким трансформер всеже показывается себя лучше.